In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from keras.datasets import mnist
from tensorflow.keras import metrics
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [ ]:

### DEFINE MODEL ###

model = Sequential()
model.add(Conv2D(256, (3, 3), activation = 'relu',padding='same', input_shape = (28, 28,1)))
model.add(Conv2D(512, (5, 5),padding='same', activation = 'relu'))
model.add(Conv2D(256, (3, 3),padding='same', activation = 'relu'))
model.add(Flatten())
model.add(Dense(784,activation='softmax'))

model.summary()



### CUSTOM LOSS - FOCAL LOSS

def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        gamma=2.
        alpha=.25
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

### METRICS ###

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

### CALLBACKS HERE ###

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,patience=2, min_lr=0.000000001)

class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    print(np.sum(self.model.predict(x_train[0].reshape(1,28,28,1))))
    display([x.reshape(28,28),x_label.reshape(28,28),self.model.predict(x.reshape(1,28,28,1)).reshape(28,28)])
 
checkpoint = ModelCheckpoint("model.h5py", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq='epoch',period=1)

### MODEL PARAMETERS ###
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,
    beta_2=0.999)

lr_metric = get_lr_metric(optimizer)
                              
#if exists('model.h5py'):
#  model.load_weights('model.h5py')
# print('loaded stuff')

model.compile(optimizer = optimizer,
              loss = binary_focal_loss_fixed,
              metrics = ['accuracy',lr_metric])

history = model.fit(x_train, y_train, epochs = 5, batch_size = 512,callbacks=[DisplayCallback(),reduce_lr,checkpoint],validation_data=(x_val, y_val))